In [1]:
import numpy as np
import pandas as pd
import pandas_ta as ta
from scipy.signal import argrelextrema
from numba import jit
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression, RFE
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
import numpy as np
import pandas as pd
import optuna
import joblib
import json
import gc
import traceback
import os
from datetime import datetime
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import recall_score,accuracy_score,precision_score

def add_indicator_to_df(df_ta, indicator):
    if indicator is None: return
    if isinstance(indicator, pd.DataFrame):
        for col in indicator.columns: df_ta[col] = indicator[col]
    elif isinstance(indicator, pd.Series):
        colname = indicator.name if indicator.name else 'Unnamed'
        df_ta[colname] = indicator

def safe_add(df_ta, func, *args, **kwargs):
    try:
        result = func(*args, **kwargs)
        add_indicator_to_df(df_ta, result)
        return True
    except:
        return False

def add_kimchi_premium_proxy(df):
    df_kp = df.copy()
    target_coins = ['ETH', 'BTC', 'XRP', 'SOL', 'ADA', 'DOGE', 'AVAX', 'DOT']
    
    for coin in target_coins:
        krw_col = f'{coin}_Close'
        bin_col = f'{coin}_Bin_Close'
        
        if krw_col in df_kp.columns and bin_col in df_kp.columns:
            ratio_col = f'{coin}_KP_Ratio'
            df_kp[ratio_col] = df_kp[krw_col] / (df_kp[bin_col] + 1e-9)
            
            ma = df_kp[ratio_col].rolling(180).mean()
            std = df_kp[ratio_col].rolling(180).std()
            df_kp[f'{coin}_KP_Zscore'] = (df_kp[ratio_col] - ma) / (std + 1e-9)
            
            df_kp[f'{coin}_KP_Change'] = df_kp[ratio_col].pct_change()

    return df_kp

def add_funding_and_onchain_features(df):
    df_on = df.copy()
    
    if 'fundingRate' in df_on.columns:
        df_on['FR_Abs_Signal'] = df_on['fundingRate'].abs()
        df_on['FR_Trend'] = df_on['fundingRate'].rolling(6).mean()
        df_on['FR_Change'] = df_on['fundingRate'].diff()

    tvl_cols = [col for col in df_on.columns if 'tvl' in col.lower()]
    for col in tvl_cols:
        df_on[f'{col}_1d_chg'] = df_on[col].pct_change(6)
        
    return df_on

def add_cross_crypto_correlations(df):
    df_corr = df.copy()
    other_coins = ['BTC', 'XRP', 'SOL', 'ADA', 'AVAX', 'DOT']
    eth_ret = df['ETH_Close'].pct_change()
    
    for coin in other_coins:
        col_name = f'{coin}_Close'
        if col_name in df.columns:
            coin_ret = df[col_name].pct_change()
            df_corr[f'Corr_ETH_{coin}_24h'] = eth_ret.rolling(6).corr(coin_ret)
            df_corr[f'RelStr_ETH_{coin}'] = eth_ret - coin_ret
            
    return df_corr

def add_price_lag_features_first_4h(df):
    df_new = df.copy()
    close = df['ETH_Close']
    for lag_days in [1, 2, 3, 5, 10]:
        lag_periods = lag_days * 6
        df_new[f'return_lag_{lag_periods}p'] = close.pct_change(periods=lag_periods).shift(1)
    return df_new

def calculate_technical_indicators_4h(df):
    df = df.sort_values('date').reset_index(drop=True)
    df_ta = df.copy()

    close = df['ETH_Close']
    high = df.get('ETH_High', close)
    low = df.get('ETH_Low', close)
    volume = df.get('ETH_Volume', pd.Series(index=df.index, data=1))
    open_ = df.get('ETH_Open', close)

    df_ta['ATR_84'] = ta.atr(high, low, close, length=84)
    bb = ta.bbands(close, length=120, std=2)
    if bb is not None:
        df_ta['BB_WIDTH'] = bb.iloc[:, 2]

    df_ta['SMA_120'] = ta.sma(close, length=120)
    df_ta['SMA_300'] = ta.sma(close, length=300)
    df_ta['EMA_72'] = ta.ema(close, length=72)
    
    df_ta['TREND_SCORE'] = (close > df_ta['SMA_120']).astype(int) + (df_ta['SMA_120'] > df_ta['SMA_300']).astype(int)

    df_ta['RSI_84'] = ta.rsi(close, length=84)
    safe_add(df_ta, ta.macd, close, fast=72, slow=156, signal=54)

    df_ta['OBV'] = ta.obv(close, volume)
    df_ta['MFI_84'] = ta.mfi(high, low, close, volume, length=84)
    df_ta['VOLUME_RATIO'] = volume / (volume.rolling(120).mean() + 1e-8)

    df_ta['UPPER_SHADOW'] = (high - np.maximum(close, open_)) / (high - low + 1e-9)
    df_ta['LOWER_SHADOW'] = (np.minimum(close, open_) - low) / (high - low + 1e-9)

    for window in [30, 120, 360]:
        swing_high = high.rolling(window).max().shift(1)
        swing_low = low.rolling(window).min().shift(1)
        
        df_ta[f'PRICE_VS_HIGH_{window}p'] = close / (swing_high + 1e-9)
        df_ta[f'PRICE_VS_LOW_{window}p'] = close / (swing_low + 1e-9)
        df_ta[f'BREAKOUT_STR_{window}p'] = (close - swing_high) / (df_ta['ATR_84'] + 1e-9)

    return df_ta

def add_volatility_regime_features_4h(df):
    df_regime = df.copy()
    if 'ATR_84' in df.columns:
        atr_ma = df['ATR_84'].rolling(120).mean()
        df_regime['high_volatility_regime'] = (df['ATR_84'] > atr_ma).astype(int)
    return df_regime

def calculate_cumulative_volume_delta_4h(df):
    df_cvd = df.copy()
    close = df['ETH_Close']
    open_ = df.get('ETH_Open', close)
    volume = df.get('ETH_Volume', pd.Series(index=df.index, data=1))
    
    df_cvd['volume_delta'] = np.where(close > open_, volume, -volume)
    df_cvd['CVD'] = df_cvd['volume_delta'].cumsum()
    
    df_cvd['CVD_24h'] = df_cvd['volume_delta'].rolling(6).sum()
    df_cvd['CVD_7d'] = df_cvd['volume_delta'].rolling(42).sum()
    
    df_cvd['CVD_Rank_180'] = df_cvd['CVD_24h'].rolling(180).rank(pct=True)
    
    cvd_slope = df_cvd['CVD'].diff(6)
    price_volatility = close.rolling(24).std() + 1e-9
    df_cvd['CVD_Slope_Norm'] = cvd_slope / price_volatility 
    
    return df_cvd

def add_vwma_features_4h(df):
    df_vwma = df.copy()
    close = df['ETH_Close']
    volume = df.get('ETH_Volume', pd.Series(index=df.index, data=1))
    
    for period in [20, 50]:
        period_4h = period * 6
        vwma = (close * volume).rolling(period_4h).sum() / (volume.rolling(period_4h).sum() + 1e-9)
        df_vwma[f'Price_div_VWMA_{period}d'] = (close / (vwma + 1e-9)) - 1
        
    return df_vwma

def add_obv_divergence_features_4h(df):
    df_obv = df.copy()
    close = df['ETH_Close']
    volume = df.get('ETH_Volume', pd.Series(index=df.index, data=1))
    
    if 'OBV' not in df.columns:
        df_obv['OBV'] = ta.obv(close, volume)
    obv = df_obv['OBV']
    
    obv_ma_30 = obv.rolling(30).mean()
    obv_ma_60 = obv.rolling(60).mean()
    df_obv['OBV_Trend'] = np.where(obv_ma_30 > obv_ma_60, 1, -1)
    
    def is_pivot_high(series, window=2):
        center = series.shift(window)
        is_max = pd.Series(True, index=series.index)
        for i in range(1, window + 1):
            is_max &= (center > series.shift(window - i)) & (center > series.shift(window + i))
        return is_max

    def is_pivot_low(series, window=2):
        center = series.shift(window)
        is_min = pd.Series(True, index=series.index)
        for i in range(1, window + 1):
            is_min &= (center < series.shift(window - i)) & (center < series.shift(window + i))
        return is_min

    price_high = is_pivot_high(close, 2)
    obv_high = is_pivot_high(obv, 2)
    
    price_slope = close.diff(12)
    obv_slope = obv.diff(12)
    
    df_obv['Div_Bullish'] = ((price_slope < 0) & (obv_slope > 0)).astype(int)
    df_obv['Div_Bearish'] = ((price_slope > 0) & (obv_slope < 0)).astype(int)
    
    return df_obv

def add_vwap_anchored_features_4h(df):
    df_vwap = df.copy()
    

    if 'date' not in df_vwap.columns and 'timestamp' in df_vwap.columns:
        df_vwap['date'] = pd.to_datetime(df_vwap['timestamp'])
    else:
        df_vwap['date'] = pd.to_datetime(df_vwap['date'])
    

    close = df_vwap['ETH_Close']
    high = df_vwap.get('ETH_High', close)
    low = df_vwap.get('ETH_Low', close)
    

    if 'ETH_Volume' in df_vwap.columns:
        volume = df_vwap['ETH_Volume']
    elif 'volume' in df_vwap.columns: 
        volume = df_vwap['volume']
    else:
        volume = pd.Series(1, index=df_vwap.index)

    typical_price = (high + low + close) / 3
    
    # 3. VWAP 계산 로직
    df_vwap['day'] = df_vwap['date'].dt.date
    

    df_vwap['pv'] = typical_price * volume
    df_vwap['cum_pv'] = df_vwap.groupby('day')['pv'].cumsum()
    df_vwap['cum_vol'] = volume.groupby(df_vwap['day']).cumsum() 
    
    df_vwap['VWAP_Day'] = df_vwap['cum_pv'] / (df_vwap['cum_vol'] + 1e-9)
    df_vwap['Dist_from_VWAP'] = (close / df_vwap['VWAP_Day']) - 1
    
    # 임시 컬럼 제거
    df_vwap = df_vwap.drop(['day', 'pv', 'cum_pv', 'cum_vol'], axis=1, errors='ignore')
    
    return df_vwap

def add_institutional_footprint_features_4h(df):
    df_inst = df.copy()
    volume = df.get('ETH_Volume', pd.Series(index=df_inst.index, data=1))
    close = df_inst['ETH_Close']
    open_ = df_inst.get('ETH_Open', close)
    high = df_inst.get('ETH_High', close)
    low = df_inst.get('ETH_Low', close)
    
    vol_ma = volume.rolling(30).mean()
    df_inst['Vol_Spike'] = (volume > (vol_ma * 2.0)).astype(int)
    
    body_size = (close - open_).abs()
    candle_range = high - low
    df_inst['Small_Body'] = (body_size < (candle_range * 0.3))
    
    df_inst['Absorption'] = (df_inst['Vol_Spike'] & df_inst['Small_Body']).astype(int)
    
    lower_wick = (open_.combine(close, min) - low)
    upper_wick = (high - open_.combine(close, max))
    
    df_inst['Buying_Climax'] = (df_inst['Vol_Spike'] & (upper_wick > body_size * 2)).astype(int)
    df_inst['Selling_Climax'] = (df_inst['Vol_Spike'] & (lower_wick > body_size * 2)).astype(int)
    
    return df_inst


def preprocess_non_stationary_features_4h(df):
    df_proc = df.copy()
    
    # 1. 접두사 매칭 (기존)
    prefixes_to_transform = [
        'eth_', 'aave_', 'lido_', 'makerdao_', 'uniswap_', 'curve_', 'chain_',
        'l2_', 'usdt_', 'total'
    ]
    
    # 2. [핵심 수정] 거시경제 지표 "정확한 이름" 추가 (언더바 여부 상관없이)
    exact_cols_to_transform = ['SP500', 'GOLD', 'DXY', 'VIX', 'sp500', 'gold', 'dxy', 'vix']

    cols_to_transform = []
    for col in df_proc.columns:
        col_lower = col.lower()
        
        # (A) 정확한 이름 매칭 (거시경제 지표용)
        if col in exact_cols_to_transform:
            cols_to_transform.append(col)
            continue # 이미 찾았으니 다음 루프로

        # (B) 접두사 매칭 (기존 로직)
        if col.startswith(tuple(prefixes_to_transform)):
            exclude_prefixes = ['fg_', 'funding_', 'open', 'high', 'low', 'close', 'volume']
            exclude_keywords = ['_pct_', '_ratio', 'target', 'next_', 'date']
            
            if not col.startswith(tuple(exclude_prefixes)):
                if not any(k in col_lower for k in exclude_keywords):
                    cols_to_transform.append(col)
    
    # 중복 제거
    cols_to_transform = list(set(cols_to_transform))
    
    # 3. 변환 로직 (기존과 동일: 24시간 변화율 & MA 괴리율)
    cols_to_drop = []
    for col in cols_to_transform:
        series = df_proc[col].fillna(method='ffill').replace(0, 1e-9)
        
        # 24시간 전 대비 변화율 (Frozen 해결사)
        df_proc[f'{col}_pct_chg_24h'] = series.pct_change(6)
        
        # 장기 이동평균 괴리율
        ma_180 = series.rolling(window=180, min_periods=60).mean()
        df_proc[f'{col}_ma180_ratio'] = (series - ma_180) / (ma_180 + 1e-9)
        
        cols_to_drop.append(col) # 원본(Frozen된 놈)은 삭제

    df_proc = df_proc.drop(columns=cols_to_drop, errors='ignore')
    df_proc = df_proc.replace([np.inf, -np.inf], 0).fillna(0)
    
    return df_proc


def remove_raw_prices_and_transform(df, target_type, method):
    df_transformed = df.copy()
    
    if 'eth_log_return' not in df_transformed.columns:
        df_transformed['eth_log_return'] = np.log(df['ETH_Close'] / df['ETH_Close'].shift(1))
    
    remove_patterns = ['_Close', '_Open', '_High', '_Low', '_Volume']
    keep_keywords = [
        '_lag', '_position', '_ratio', '_range', '_change', '_corr', '_volatility', '_obv',
        'PRICE_VS', 'BREAKOUT', 'UPPER_SHADOW', 'LOWER_SHADOW', 'BB_WIDTH', 'KP', 'FR', 'RelStr',
        'CVD', 'VWMA', 'VWAP', 'Div', 'Spike', 'Climax', 'Absorption'
    ]
    
    cols_to_remove = [
        col for col in df_transformed.columns
        if any(p in col for p in remove_patterns)
        and not any(d in col.lower() for d in [k.lower() for k in keep_keywords])
    ]
    df_transformed.drop(cols_to_remove, axis=1, inplace=True)
    return df_transformed




2025-11-27 20:42:41.491725: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-27 20:42:41.491770: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-27 20:42:41.493224: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-27 20:42:41.500590: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-27 20:42:42.343808: W tensorflow/comp

In [2]:
@jit(nopython=True)
def compute_targets_with_hourly_numba_4h(
    base_dates_ts, base_atr, hourly_dates_ts, hourly_open,
    hourly_high, hourly_low, lookahead_periods, profit_mult, stop_mult
):
    n = len(base_dates_ts)
    # 초기화: 기본값은 -1 (유효하지 않음) 또는 NaN
    targets = np.full(n, -1, dtype=np.int32)
    entry_prices = np.full(n, np.nan, dtype=np.float64) # 명시적 float64
    tp_prices = np.full(n, np.nan, dtype=np.float64)
    sl_prices = np.full(n, np.nan, dtype=np.float64)
    
    four_hour_ms = 14400000
    lookahead_ms = lookahead_periods * four_hour_ms
    MIN_PROFIT_THRESHOLD = 0.0025 
    
    h_start = 0
    
    # 마지막 데이터는 lookahead 계산 불가하므로 n-1까지만 루프
    for i in range(n - 1):
        atr = base_atr[i]
        # ATR이 없거나 0이면 계산 불가 -> continue (초기값 -1/NaN 유지)
        if np.isnan(atr) or atr <= 0: 
            continue
        
        entry_start_ts = base_dates_ts[i] + four_hour_ms
        entry_end_ts = entry_start_ts + lookahead_ms
        
        first_entry_idx = -1
        
        # 1시간봉 매칭 (h_start부터 검색하여 속도 최적화)
        for h in range(h_start, len(hourly_dates_ts)):
            if hourly_dates_ts[h] >= entry_start_ts:
                first_entry_idx = h
                h_start = h # 다음 루프를 위해 시작점 업데이트
                break
        
        # 매칭되는 1시간봉이 없으면(데이터 끝부분 등) 계산 불가 -> continue
        if first_entry_idx == -1:
            continue
        
        # 진입 가격 확정
        entry_price = hourly_open[first_entry_idx]
        tp = entry_price + (atr * profit_mult)
        sl = entry_price - (atr * stop_mult)
        
        # [중요] 배열에 값 할당 (i 인덱스 위치에 정확히!)
        entry_prices[i] = entry_price
        tp_prices[i] = tp
        sl_prices[i] = sl
        
        result = 0 # 기본적으로 실패(0)로 시작
        final_idx = -1 
        
        for h in range(first_entry_idx, len(hourly_dates_ts)):
            # 1. 시간 초과 (Lookahead 기간 종료)
            if hourly_dates_ts[h] >= entry_end_ts:
                final_idx = h
                break
            
            # 2. 손절 (Stop Loss) - 최우선 확인
            if hourly_low[h] <= sl:
                result = 0
                final_idx = -1 # 손절 당했으므로 Timeout 수익 체크 불필요
                break
                
            # 3. 익절 (Take Profit)
            if hourly_high[h] >= tp:
                result = 1
                final_idx = -1 # 익절 했으므로 종료
                break
        
        # 4. Timeout 처리 (아직 포지션 보유 중일 때 수익률 체크)
        if final_idx != -1:
            exit_price = hourly_open[final_idx]
            return_rate = (exit_price - entry_price) / entry_price
            
            if return_rate >= MIN_PROFIT_THRESHOLD:
                result = 1
            else:
                result = 0

        # 결과 할당
        targets[i] = result
    
    return targets, entry_prices, tp_prices, sl_prices


def create_targets_4h(df_4h, df_1h, lookahead=30, profit_mult=1.5, stop_mult=1.0, **kwargs):
    df_target = df_4h.copy()
    hourly_df = df_1h.copy()
    
    df_target['date'] = pd.to_datetime(df_target['date'])
    hourly_df['datetime'] = pd.to_datetime(hourly_df['datetime'])
    
    hourly_df.columns = hourly_df.columns.str.lower()
    
    if 'ATR_84' not in df_target.columns:
        df_target['ATR_84'] = ta.atr(
            df_target['ETH_High'], df_target['ETH_Low'], df_target['ETH_Close'], length=84
        )
    
    df_target = df_target.sort_values('date').reset_index(drop=True)
    hourly_df = hourly_df.sort_values('datetime').reset_index(drop=True)
    
    base_dates_ts = df_target['date'].astype(np.int64).values // 10**6
    base_atr = df_target['ATR_84'].fillna(method='ffill').fillna(0).to_numpy()
    
    hourly_dates_ts = hourly_df['datetime'].astype(np.int64).values // 10**6
    hourly_open = hourly_df['open'].astype(np.float64).to_numpy()
    hourly_high = hourly_df['high'].astype(np.float64).to_numpy()
    hourly_low = hourly_df['low'].astype(np.float64).to_numpy()
    
    targets, entry_prices, tp_prices, sl_prices = compute_targets_with_hourly_numba_4h(
        base_dates_ts, base_atr, hourly_dates_ts, hourly_open, hourly_high, hourly_low,
        lookahead, profit_mult, stop_mult
    )
    
    df_target['next_direction'] = targets
    df_target['real_entry_price'] = entry_prices
    df_target['take_profit_price'] = tp_prices
    df_target['stop_loss_price'] = sl_prices
    
    df_target['next_close'] = df_target['ETH_Close'].shift(-1)
    df_target['next_open'] = df_target['ETH_Open'].shift(-1)
    df_target['next_log_return'] = np.log(df_target['next_close'] / (df_target['next_open'] + 1e-9))
    
    if lookahead > 0:
        df_target = df_target.iloc[:-lookahead]
    
    valid_before = len(df_target)
    df_target = df_target[df_target['next_direction'] != -1].reset_index(drop=True)
    valid_after = len(df_target)
    
    print(f"Valid Samples: {valid_after}/{valid_before} (Removed: {valid_before - valid_after})")
    
    return df_target


In [3]:
def select_features_verified(X_train, y_train, task='class', top_n=20, verbose=True):
    if len(X_train) > 10000:
        idx = np.random.choice(len(X_train), 10000, replace=False)
        X_sub = X_train.iloc[idx]
        y_sub = y_train.iloc[idx]
    else:
        X_sub, y_sub = X_train, y_train

    if task == 'class':
        mi_scores = mutual_info_classif(X_sub, y_sub, random_state=42, n_neighbors=3)
    else:
        mi_scores = mutual_info_regression(X_sub, y_sub, random_state=42, n_neighbors=3)
    mi_idx = np.argsort(mi_scores)[::-1][:top_n]
    mi_features = X_train.columns[mi_idx].tolist()
    
    estimator = LGBMClassifier(n_estimators=100, random_state=42, verbose=-1) if task == 'class' else LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)
    rfe = RFE(estimator=estimator, n_features_to_select=top_n, step=0.1, verbose=0)
    rfe.fit(X_sub, y_sub)
    rfe_features = X_train.columns[rfe.support_].tolist()

    rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1) if task == 'class' else RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X_sub, y_sub)
    rf_idx = np.argsort(rf_model.feature_importances_)[::-1][:top_n]
    rf_features = X_train.columns[rf_idx].tolist()
    
    all_features = mi_features + rfe_features + rf_features
    feature_votes = Counter(all_features)
    selected_features = [feat for feat, _ in feature_votes.most_common(top_n)]
    
    if len(selected_features) < top_n:
        remaining = top_n - len(selected_features)
        for feat in mi_features:
            if feat not in selected_features:
                selected_features.append(feat)
                remaining -= 1
                if remaining == 0: break
    
    return selected_features, {}

def select_features_multi_target(X_train, y_train, target_type='direction', top_n=20):
    atr_col_name = 'ATR_84'
    if target_type == 'direction':
        selected, stats = select_features_verified(X_train, y_train['next_direction'], task='class', top_n=top_n)
        
        if atr_col_name not in selected and atr_col_name in X_train.columns:
            if len(selected) > 0: selected.pop()
            selected.insert(0, atr_col_name)
            
    print(f"\n[Feature Selection] Top {len(selected)} Features Selected:")
    print(f" -> {', '.join(selected)}")
    return selected, stats

def process_single_split(split_data, target_type='direction', top_n=20, fold_idx=None, atr_col_name='ATR_84'): 
    train_df = split_data['train'] 
    val_df = split_data['val'] 
    test_df = split_data['test'] 
    fold_type = split_data.get('fold_type', 'unknown')

    print(f"\n{'='*80}")
    print(f" Processing Fold {fold_idx} ({fold_type})")
    print(f"{'='*80}")
    print(f" Train Period: {train_df['date'].min()} ~ {train_df['date'].max()} (N={len(train_df)})")
    print(f" Val   Period: {val_df['date'].min()} ~ {val_df['date'].max()} (N={len(val_df)})")
    print(f" Test  Period: {test_df['date'].min()} ~ {test_df['date'].max()} (N={len(test_df)})")

    target_cols = [
        'next_direction', 'next_log_return', 'next_close', 'next_open', 
        'take_profit_price', 'stop_loss_price', 
        'ATR_84', 'real_entry_price' 
    ]

    train_processed = train_df.dropna(subset=target_cols).reset_index(drop=True)
    val_processed = val_df.dropna(subset=target_cols).reset_index(drop=True)
    test_processed = test_df.dropna(subset=target_cols).reset_index(drop=True)

    exclude_cols = [col for col in target_cols if col != atr_col_name] + ['date']
    feature_cols = [col for col in train_processed.columns if col not in exclude_cols]
    
    X_train = train_processed[feature_cols]
    y_train = train_processed[target_cols]
    X_val = val_processed[feature_cols]
    y_val = val_processed[target_cols]
    X_test = test_processed[feature_cols]
    y_test = test_processed[target_cols]

    balance = y_train['next_direction'].value_counts(normalize=True).to_dict()
    print(f"[Class Balance] Train Set: {balance}")

    selected_features, selection_stats = select_features_multi_target(
        X_train, y_train, target_type=target_type, top_n=top_n
    )

    X_train_sel = X_train[selected_features]
    X_val_sel = X_val[selected_features]
    X_test_sel = X_test[selected_features]

    scaler = MinMaxScaler(feature_range=(-1, 1))
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_sel), columns=selected_features)
    X_val_scaled = pd.DataFrame(scaler.transform(X_val_sel), columns=selected_features)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test_sel), columns=selected_features)

    return {
        'train': {'X': X_train_scaled, 'y': y_train.reset_index(drop=True), 'dates': train_processed['date'].reset_index(drop=True)},
        'val': {'X': X_val_scaled, 'y': y_val.reset_index(drop=True), 'dates': val_processed['date'].reset_index(drop=True)},
        'test': {'X': X_test_scaled, 'y': y_test.reset_index(drop=True), 'dates': test_processed['date'].reset_index(drop=True)},
        'scaler': scaler, 
        'selected_features': selected_features,
        'stats': {
            'fold_idx': fold_idx if fold_idx is not None else split_data.get('fold_idx', 0),
            'fold_type': split_data.get('fold_type', 'unknown')
        }
    }

def split_walk_forward_method(df, train_start_date, final_test_start='2025-01-01', 
                              initial_train_size=800, val_size=150, test_size=150, 
                              step=150, gap_size=30):
    
    df_period = df[df['date'] >= pd.to_datetime(train_start_date)].copy()
    df_period = df_period.sort_values('date').reset_index(drop=True)
    
    if isinstance(final_test_start, str):
        final_test_start = pd.to_datetime(final_test_start)
    
    total_len = len(df_period)
    folds = []
    current_test_end = total_len
    
    while True:
        test_end_idx = current_test_end
        test_start_idx = test_end_idx - test_size
        val_end_idx = test_start_idx - gap_size
        val_start_idx = val_end_idx - val_size
        train_end_idx = val_start_idx - gap_size
        train_start_idx = train_end_idx - initial_train_size
        
        if train_start_idx < 0: break
        
        train_fold = df_period.iloc[train_start_idx:train_end_idx].copy()
        val_fold = df_period.iloc[val_start_idx:val_end_idx].copy()
        test_fold = df_period.iloc[test_start_idx:test_end_idx].copy()
        
        folds.append({
            'train': train_fold,
            'val': val_fold,
            'test': test_fold,
            'fold_type': 'walk_forward_rolling_reverse'
        })
        current_test_end = test_start_idx - gap_size
    
    folds.reverse()
    for idx, fold in enumerate(folds):
        fold['fold_idx'] = idx + 1
        
    final_test_df = df_period[df_period['date'] >= final_test_start].copy()
    if len(final_test_df) > 0:
        pre_final_df = df_period[df_period['date'] < final_test_start].copy()
        if len(pre_final_df) >= (initial_train_size + val_size + gap_size):
            final_val_end_idx = len(pre_final_df)
            final_val_start_idx = final_val_end_idx - val_size
            final_train_end_idx = final_val_start_idx - gap_size
            final_train_start_idx = final_train_end_idx - initial_train_size
            
            final_train_data = pre_final_df.iloc[final_train_start_idx:final_train_end_idx].copy()
            final_val_data = pre_final_df.iloc[final_val_start_idx:final_val_end_idx].copy()
            
            folds.append({
                'train': final_train_data,
                'val': final_val_data,
                'test': final_test_df,
                'fold_idx': len(folds) + 1,
                'fold_type': 'final_holdout'
            })
            
    return folds

def build_complete_pipeline_corrected(df_raw, df_hour, train_start_date, **kwargs): 
    print(f"\n Pipeline Started... (Train Start: {train_start_date})")

    df = df_raw.copy()
    
    df = add_kimchi_premium_proxy(df)
    df = add_funding_and_onchain_features(df)
    df = calculate_technical_indicators_4h(df)
    df = calculate_cumulative_volume_delta_4h(df)
    df = add_vwma_features_4h(df)
    df = add_obv_divergence_features_4h(df)
    df = add_vwap_anchored_features_4h(df)
    df = add_institutional_footprint_features_4h(df)
    
    df = add_price_lag_features_first_4h(df)
    df = add_cross_crypto_correlations(df)
    df = add_volatility_regime_features_4h(df)
    df = preprocess_non_stationary_features_4h(df)
    
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)

    lookahead = kwargs.get('lookahead_periods', 30) 
    profit_mult = kwargs.get('profit_mult', 2.0)
    stop_mult = kwargs.get('stop_mult', 1.0)

    df = create_targets_4h(df, df_hour,
        lookahead=lookahead, 
        profit_mult=profit_mult, 
        stop_mult=stop_mult
    )

    df = remove_raw_prices_and_transform(df, 'direction', 'tvt')
    print(f"Final Data Shape: {df.shape}")
    
    initial_train_size = kwargs.get('initial_train_days', 800) * 6
    val_size = 150 * 6
    test_size = 150 * 6
    gap_size = lookahead

    splits = split_walk_forward_method(
        df, 
        train_start_date=train_start_date,
        final_test_start=kwargs.get('final_test_start', '2025-01-01'),
        initial_train_size=initial_train_size,
        val_size=val_size,
        test_size=test_size,
        step=150 * 6,
        gap_size=gap_size
    )
    print(f"Data Split Completed. Total {len(splits)} folds generated.")

    result = []
    for fold in splits:
        res = process_single_split(
            fold, 
            top_n=kwargs.get('top_n', 20), 
            fold_idx=fold['fold_idx'],
            atr_col_name='ATR_84'
        )
        result.append(res)

    return result

In [4]:
import numpy as np
import pandas as pd
import optuna
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, accuracy_score
from sklearn.utils.class_weight import compute_class_weight

class DirectionModels:
    
    @staticmethod
    def get_class_weights(y_train):
        classes = np.unique(y_train)
        weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
        return dict(zip(classes, weights))

    @staticmethod
    def _calculate_expectancy(y_true, y_prob, th_start=0.5, th_end=0.9, th_step=0.05, rr_ratio=1.5):
        """
        최적의 Expectancy를 찾아 반환하는 내부 헬퍼 함수
        """
        best_exp = -999.0
        best_th = 0.5
        best_metrics = {'win_rate': 0.0, 'trades': 0}
        
        for th in np.arange(th_start, th_end, th_step):
            preds = (y_prob >= th).astype(int)
            n_trades = preds.sum()
            
            if n_trades < 50: continue # 최소 거래 횟수 필터
            
            wins = ((preds == 1) & (y_true == 1)).sum()
            losses = n_trades - wins
            
            # Expectancy = (승률 * 수익비) - (패율 * 손실비)
            # 여기서는 손실비=1.0 가정
            exp = (wins * rr_ratio - losses * 1.0) / n_trades
            
            if exp > best_exp:
                best_exp = exp
                best_th = th
                best_metrics = {'win_rate': wins/n_trades, 'trades': n_trades}
                
        return best_exp, best_th, best_metrics

    @staticmethod
    def random_forest(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 300, 1000),
                'max_depth': trial.suggest_int('max_depth', 4, 12),
                'min_samples_split': trial.suggest_int('min_samples_split', 20, 150),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 60),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
                'ccp_alpha': trial.suggest_float('ccp_alpha', 1e-4, 1e-2, log=True),
                'class_weight': 'balanced',
                'random_state': 42,
                'n_jobs': -1
            }
            
            model = RandomForestClassifier(**params)
            model.fit(X_train, y_train)
            
            # [수정] Expectancy 최적화
            val_prob = model.predict_proba(X_val)[:, 1]
            exp, _, _ = DirectionModels._calculate_expectancy(y_val, val_prob)
            return exp

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=20)
        
        best_model = RandomForestClassifier(**study.best_params, class_weight='balanced', random_state=42, n_jobs=-1)
        best_model.fit(X_train, y_train)
        
        # [수정] 최종 평가 및 출력
        train_prob = best_model.predict_proba(X_train)[:, 1]
        val_prob = best_model.predict_proba(X_val)[:, 1]
        
        train_exp, _, _ = DirectionModels._calculate_expectancy(y_train, train_prob)
        val_exp, val_th, val_meta = DirectionModels._calculate_expectancy(y_val, val_prob)
        
        print(f"  [RandomForest] Train Exp: {train_exp:.4f}R | Val Exp: {val_exp:.4f}R (WinRate: {val_meta['win_rate']*100:.1f}%, Trades: {val_meta['trades']}) | Gap: {train_exp - val_exp:.4f}")
        
        return best_model

    @staticmethod
    def lightgbm(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            params = {
                'n_estimators': 1000,
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 20, 100),
                'max_depth': trial.suggest_int('max_depth', 4, 10),
                'min_child_samples': trial.suggest_int('min_child_samples', 30, 150),
                'subsample': trial.suggest_float('subsample', 0.5, 0.9),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.9),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-2, 10.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-2, 10.0, log=True),
                'objective': 'binary',
                'metric': 'binary_logloss',
                'class_weight': 'balanced',
                'verbosity': -1,
                'n_jobs': -1,
                'random_state': 42
            }
            
            model = lgb.LGBMClassifier(**params)
            callbacks = [lgb.early_stopping(stopping_rounds=30, verbose=False)]
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=callbacks)
            
            # [수정] Expectancy 최적화
            val_prob = model.predict_proba(X_val)[:, 1]
            exp, _, _ = DirectionModels._calculate_expectancy(y_val, val_prob)
            return exp

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=25)
        
        best_params = study.best_params
        best_params.update({
            'n_estimators': 1000, 
            'objective': 'binary',
            'metric': 'binary_logloss', 
            'class_weight': 'balanced', 
            'verbosity': -1, 
            'n_jobs': -1, 
            'random_state': 42
        })
        
        final_model = lgb.LGBMClassifier(**best_params)
        final_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)])
        
        # [수정] 최종 평가 및 출력
        train_prob = final_model.predict_proba(X_train)[:, 1]
        val_prob = final_model.predict_proba(X_val)[:, 1]
        
        train_exp, _, _ = DirectionModels._calculate_expectancy(y_train, train_prob)
        val_exp, val_th, val_meta = DirectionModels._calculate_expectancy(y_val, val_prob)
        
        print(f"  [LightGBM]     Train Exp: {train_exp:.4f}R | Val Exp: {val_exp:.4f}R (WinRate: {val_meta['win_rate']*100:.1f}%, Trades: {val_meta['trades']}) | Gap: {train_exp - val_exp:.4f}")
        
        return final_model

    @staticmethod
    def xgboost(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            params = {
                'n_estimators': 1000,
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'max_depth': trial.suggest_int('max_depth', 4, 10),
                'min_child_weight': trial.suggest_int('min_child_weight', 2, 15),
                'subsample': trial.suggest_float('subsample', 0.5, 0.9),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.9),
                'gamma': trial.suggest_float('gamma', 0.5, 10.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-2, 10.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-2, 10.0, log=True),
                'objective': 'binary:logistic',
                'eval_metric': 'logloss',
                'tree_method': 'hist',
                'early_stopping_rounds': 30,
                'random_state': 42,
                'n_jobs': -1
            }
            
            model = xgb.XGBClassifier(**params)
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
            
            # [수정] Expectancy 최적화
            val_prob = model.predict_proba(X_val)[:, 1]
            exp, _, _ = DirectionModels._calculate_expectancy(y_val, val_prob)
            return exp

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=25)
        
        best_params = study.best_params
        best_params.update({
            'n_estimators': 1000, 
            'objective': 'binary:logistic',
            'eval_metric': 'logloss', 
            'tree_method': 'hist', 
            'early_stopping_rounds': 50,
            'random_state': 42, 
            'n_jobs': -1
        })
        
        final_model = xgb.XGBClassifier(**best_params)
        final_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        
        # [수정] 최종 평가 및 출력
        train_prob = final_model.predict_proba(X_train)[:, 1]
        val_prob = final_model.predict_proba(X_val)[:, 1]
        
        train_exp, _, _ = DirectionModels._calculate_expectancy(y_train, train_prob)
        val_exp, val_th, val_meta = DirectionModels._calculate_expectancy(y_val, val_prob)
        
        print(f"  [XGBoost]      Train Exp: {train_exp:.4f}R | Val Exp: {val_exp:.4f}R (WinRate: {val_meta['win_rate']*100:.1f}%, Trades: {val_meta['trades']}) | Gap: {train_exp - val_exp:.4f}")
        
        return final_model

    @staticmethod
    def catboost(X_train, y_train, X_val, y_val):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            params = {
                'iterations': 1000,
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'depth': trial.suggest_int('depth', 4, 9),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 2, 15),
                'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
                'loss_function': 'Logloss',
                'eval_metric': 'Logloss',
                'auto_class_weights': 'Balanced',
                'logging_level': 'Silent',
                'random_seed': 42,
                'od_type': 'Iter',
                'od_wait': 30,
                'allow_writing_files': False
            }
            
            model = cb.CatBoostClassifier(**params)
            model.fit(X_train, y_train, eval_set=(X_val, y_val))
            
            val_prob = model.predict_proba(X_val)[:, 1]
            exp, _, _ = DirectionModels._calculate_expectancy(y_val, val_prob)
            return exp

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=20)
        
        best_params = study.best_params
        best_params.update({
            'iterations': 1000, 
            'loss_function': 'Logloss', 
            'eval_metric': 'Logloss',
            'auto_class_weights': 'Balanced', 
            'logging_level': 'Silent',
            'random_seed': 42, 
            'od_type': 'Iter', 
            'od_wait': 50, 
            'allow_writing_files': False
        })
        
        final_model = cb.CatBoostClassifier(**best_params)
        final_model.fit(X_train, y_train, eval_set=(X_val, y_val))
        
        train_prob = final_model.predict_proba(X_train)[:, 1]
        val_prob = final_model.predict_proba(X_val)[:, 1]
        
        train_exp, _, _ = DirectionModels._calculate_expectancy(y_train, train_prob)
        val_exp, val_th, val_meta = DirectionModels._calculate_expectancy(y_val, val_prob)
        
        print(f"  [CatBoost]     Train Exp: {train_exp:.4f}R | Val Exp: {val_exp:.4f}R (WinRate: {val_meta['win_rate']*100:.1f}%, Trades: {val_meta['trades']}) | Gap: {train_exp - val_exp:.4f}")
        
        return final_model


    
        
ML_MODELS_CLASSIFICATION = [
    {'index': 1, 'name': 'CatBoost', 'func': DirectionModels.catboost, 'needs_val': True},
    {'index': 2, 'name': 'RandomForest', 'func': DirectionModels.random_forest, 'needs_val': True},
    {'index': 3, 'name': 'LightGBM', 'func': DirectionModels.lightgbm, 'needs_val': True},
    {'index': 4, 'name': 'XGBoost', 'func': DirectionModels.xgboost, 'needs_val': True}
]

In [5]:
class ModelEvaluator:
    def __init__(self, save_models=False):
        self.results = []
        self.best_thresholds = {}
        self.save_models = save_models
        self.models = {} if save_models else None
        self.prediction_logs = {} 

    def optimize_threshold(self, y_true, buy_prob, min_trades=50, reward_risk_ratio=1.5):
        
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            th = trial.suggest_float('threshold', 0.5, 0.9)
            preds = (buy_prob >= th).astype(int)
            n_trades = np.sum(preds == 1)
            
            if n_trades < min_trades: return -999.0
            
            wins = np.sum((preds == 1) & (y_true == 1))
            losses = n_trades - wins
            
            expectancy = ((wins * reward_risk_ratio) - (losses * 1.0)) / n_trades
            if (wins / n_trades) < 0.4: expectancy -= 0.5
            
            return expectancy

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=20)
        return study.best_params['threshold']

    def evaluate_model(self, model, X_train, y_train, X_val, y_val, X_test, y_test, 
                       model_name, profit_mult=1.5, stop_mult=1.0,
                       test_dates=None, test_prices=None):
        
        rr_ratio = profit_mult / stop_mult if stop_mult > 0 else 1.5
        
        val_prob = model.predict_proba(X_val)[:, 1]
        test_prob = model.predict_proba(X_test)[:, 1]
        

        best_th = self.optimize_threshold(y_val.astype(int), val_prob, min_trades=50, reward_risk_ratio=rr_ratio)
        self.best_thresholds[model_name] = best_th
        
        # Test 셋 평가
        test_preds = (test_prob >= best_th).astype(int)
        n_trades = np.sum(test_preds == 1)
        y_test_arr = y_test.astype(int)
        
        if n_trades > 0:
            wins = np.sum((test_preds == 1) & (y_test_arr == 1))
            losses = n_trades - wins
            win_rate = wins / n_trades
            expectancy = ((wins * rr_ratio) - (losses * 1.0)) / n_trades
        else:
            win_rate, expectancy = 0.0, 0.0
            
        # [추가] 평가 결과 콘솔 출력 (한눈에 확인용)
        print(f"    -> [TEST EVAL] {model_name:<12} | Exp: {expectancy:.4f}R | Win: {win_rate*100:.1f}% | Trades: {n_trades} | Th: {best_th:.2f}")

        if test_dates is not None:
            pred_df = pd.DataFrame({
                'timestamp': test_dates,
                'close': test_prices if test_prices is not None else 0,
                'prob': test_prob,
                'threshold': best_th,
                'signal': test_preds,
                'actual_target': y_test_arr
            })
            if not isinstance(pred_df.index, pd.DatetimeIndex):
                pred_df.set_index('timestamp', inplace=True)
            self.prediction_logs[model_name] = pred_df

        result = {
            'Model': model_name,
            'Threshold': best_th,
            'Test_Trades': n_trades,
            'Test_WinRate': win_rate,
            'Test_Expectancy': expectancy
        }
        self.results.append(result)
        
        if self.save_models: self.models[model_name] = model
        return result

    def get_summary_dataframe(self): return pd.DataFrame(self.results)
    def get_models_dict(self): return self.models or {}
    def get_prediction_logs(self): return self.prediction_logs
    def get_best_thresholds(self): return self.best_thresholds


class ModelTrainer:
    def __init__(self, evaluator, lookback=30):
        self.evaluator = evaluator
        self.lookback = lookback
    
    def _prepare_target(self, y_data):
        if isinstance(y_data, pd.DataFrame):
            y_data = y_data.iloc[:, 0].values
        elif isinstance(y_data, pd.Series):
            y_data = y_data.values
        y_data = np.array(y_data).flatten()
        y_data = np.nan_to_num(y_data, nan=0.0)
        return np.round(y_data).astype(int)

    def train_all_models(self, X_train, y_train, X_val, y_val, X_test, y_test, 
                         profit_mult, stop_mult, ml_models,
                         test_dates=None, test_prices=None):
        
        y_train_arr = self._prepare_target(y_train)
        y_val_arr = self._prepare_target(y_val)
        y_test_arr = self._prepare_target(y_test)
        
        for config in ml_models:
            try:
                # 모델 학습
                if config.get('needs_val', False):
                    model = config['func'](X_train, y_train_arr, X_val, y_val_arr)
                else:
                    model = config['func'](X_train, y_train_arr)
                
                # 평가 (메타 데이터 전달)
                self.evaluator.evaluate_model(
                    model, X_train, y_train_arr, X_val, y_val_arr, X_test, y_test_arr,
                    config['name'], profit_mult, stop_mult,
                    test_dates=test_dates, test_prices=test_prices 
                )
                del model
                gc.collect()
            except Exception as e:
                print(f"[Error] Failed {config['name']}: {e}")
                traceback.print_exc()    
    
    
    
    
def save_fold_results(fold_idx, fold_type, evaluator, trial_name, fold_data, result_dir):
    base_dir = f"{result_dir}/{trial_name}/fold_{fold_idx}_{fold_type}"
    os.makedirs(base_dir, exist_ok=True)
    
    # 1. 결과 요약 저장
    summary = evaluator.get_summary_dataframe()
    summary.to_csv(f"{base_dir}/fold_summary.csv", index=False)
    
    # 2. 상세 예측 로그 저장
    pred_logs = evaluator.get_prediction_logs()
    for model_name, df_log in pred_logs.items():
        df_log.to_csv(f"{base_dir}/predictions_{model_name}.csv")

    # 3. 모델 객체 저장
    for name, model in evaluator.get_models_dict().items():
        joblib.dump(model, f"{base_dir}/model_{name}.pkl")
            
    # 4. 스케일러 저장
    if 'scaler' in fold_data:
        joblib.dump(fold_data['scaler'], f"{base_dir}/scaler.pkl")
        
    # 5. [중요] 메타데이터 저장 
    meta_data = {
        'fold_idx': fold_idx,
        'fold_type': fold_type,
        'selected_features': fold_data.get('selected_features', []), 
        'model_thresholds': evaluator.get_best_thresholds(),
        'trial_params': {
            'profit_mult': fold_data.get('profit_mult'), 
            'stop_mult': fold_data.get('stop_mult'),     
            'lookahead': fold_data.get('lookahead')     
        }
    }
    
    with open(f"{base_dir}/metadata.json", 'w') as f:
        json.dump(meta_data, f, indent=4)
            
    return summary




In [6]:
def run_optuna_optimization(df_merged, df_hour, ml_models, n_trials=30):
    
    TIMESTAMP = datetime.now().strftime("%Y-%m-%d")
    RESULT_DIR = f"model_results/{TIMESTAMP}_Sniper"
    os.makedirs(RESULT_DIR, exist_ok=True)
    
    LOG_PATH = f"{RESULT_DIR}/optuna_log.csv"

    price_col = 'ETH_Close' if 'ETH_Close' in df_merged.columns else 'close'
    
    # 날짜를 인덱스로 설정하여 검색 속도 향상
    lookup_df = df_merged[['date', price_col]].copy()
    lookup_df['date'] = pd.to_datetime(lookup_df['date'])
    lookup_df = lookup_df.set_index('date').sort_index()
    
    # [Resume] 기존 로그 로드
    existing_history = pd.DataFrame()
    if os.path.exists(LOG_PATH):
        try:
            existing_history = pd.read_csv(LOG_PATH)
            print(f"\n[Resume] Loaded {len(existing_history)} existing trials.")
        except: pass

    if not os.path.exists(LOG_PATH):
        with open(LOG_PATH, "w") as f:
            f.write("trial,lookahead,profit_mult,stop_mult,top_n,train_days,score\n")

    def objective(trial):
        nonlocal existing_history 
        
        # 파라미터 제안
        lookahead = trial.suggest_int('lookahead', 3, 12, step=3)
    
        # 2. 익절
        p_mult = trial.suggest_float('profit_mult', 1.0,1.8, step=0.2)

        # 3. 손절
        s_mult = trial.suggest_float('stop_mult', 0.5, 1.0, step=0.1)

        # 4. Feature 수
        top_n = trial.suggest_int('top_n', 10, 30, step=5)

        # 5. 학습 기간
        train_days = trial.suggest_int('train_days', 180, 730, step=180)

        # 중복 실행 방지 
        if not existing_history.empty:
            mask = (
                (existing_history['lookahead'] == lookahead) &
                (np.isclose(existing_history['profit_mult'], p_mult, atol=1e-5)) &
                (np.isclose(existing_history['stop_mult'], s_mult, atol=1e-5)) &
                (existing_history['top_n'] == top_n) &
                (existing_history['train_days'] == train_days)
            )
            if mask.any():
                prev_score = existing_history.loc[mask, 'score'].values[0]
                print(f"[Skip] Already done. Score: {prev_score}")
                return prev_score

        trial_name = f"T{trial.number}_L{lookahead}_P{p_mult:.1f}_S{s_mult:.1f}_N{top_n}"
        print(f"\n{'='*60}\n Starting {trial_name}\n{'='*60}")
        
        try:
            # 1. Build Pipeline (데이터 준비)
            pipeline_result = build_complete_pipeline_corrected(
                df_raw=df_merged,
                df_hour=df_hour,
                train_start_date='2020-01-01',
                final_test_start='2025-01-01',
                lookahead_periods=lookahead,
                profit_mult=p_mult,
                stop_mult=s_mult,
                top_n=top_n,
                initial_train_days=train_days
            )
            
            fold_scores = []
            
            # 2. Walk-Forward Loop
            for fold_data in pipeline_result:
                stats = fold_data.get('stats', {}) 
                fold_idx = stats.get('fold_idx', 0)
                fold_type = stats.get('fold_type', 'unknown')
                
                print(f"   >> Running Fold {fold_idx} ({fold_type})")
                
                fold_data['profit_mult'] = p_mult
                fold_data['stop_mult'] = s_mult
                
                # =============================================================
                # [핵심] 전략팀을 위한 원본 가격 매핑 (Lookup)
                # =============================================================
                # process_single_split이 저장해둔 날짜를 가져옵니다.
                test_dates = pd.to_datetime(fold_data['test']['dates'])
                

                test_prices = lookup_df.reindex(test_dates)[price_col].values
                

                test_prices = np.nan_to_num(test_prices, nan=0.0)
                # =============================================================

                evaluator = ModelEvaluator(save_models=True) 
                trainer = ModelTrainer(evaluator)
                
                # 모델 학습 (test_dates와 test_prices를 함께 전달)
                trainer.train_all_models(
                    fold_data['train']['X'], fold_data['train']['y'],
                    fold_data['val']['X'], fold_data['val']['y'],
                    fold_data['test']['X'], fold_data['test']['y'],
                    p_mult, s_mult, ml_models,
                    test_dates=test_dates, 
                    test_prices=test_prices 
                )

                # 결과 및 모델 저장
                summary = save_fold_results(fold_idx, fold_type, evaluator, trial_name, fold_data, RESULT_DIR)
                
                # 점수 집계 (Expectancy)
                if 'Test_Expectancy' in summary.columns:
                    best_fold_score = summary['Test_Expectancy'].max()
                    fold_scores.append(best_fold_score)
                
                del evaluator, trainer
                gc.collect()
            
            final_score = np.mean(fold_scores) if fold_scores else -99.0
            print(f"\n === Trial Score: {final_score:.4f}R ===")
            
            # 로그 기록
            with open(LOG_PATH, "a") as f:
                f.write(f"{trial.number},{lookahead},{p_mult},{s_mult},{top_n},{train_days},{final_score}\n")
            
            # 메모리 DB 업데이트
            new_row = pd.DataFrame([[trial.number, lookahead, p_mult, s_mult, top_n, train_days, final_score]], 
                                   columns=['trial','lookahead','profit_mult','stop_mult','top_n','train_days','score'])
            if existing_history.empty: existing_history = new_row
            else: existing_history = pd.concat([existing_history, new_row], ignore_index=True)
                
            return final_score
            
        except Exception as e:
            print(f" [Error] Trial Failed: {e}")
            traceback.print_exc()
            return -99.0

    # Optuna 실행
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)
    
    print(f"\n[Optuna] Best Params: {study.best_params}")
    return study


In [ ]:
# -----------------------------------------------------------------------------
# 5. Main Execution Block
# -----------------------------------------------------------------------------

# 1. GPU Check (Optional)
if tf.config.list_physical_devices('GPU'):
    print(" GPU Detected!")

print("Loading Data...")
df_merged = pd.read_csv("eth_4hour.csv")
df_hour = pd.read_csv("eth_hour.csv")
if 'timestamp' in df_merged.columns:
    df_merged = df_merged.rename(columns={'timestamp': 'date'})

df_merged['date'] = pd.to_datetime(df_merged['date'])
df_merged = df_merged.sort_values('date').reset_index(drop=True)


# [실제 실행]
study = run_optuna_optimization(df_merged, df_hour, ML_MODELS_CLASSIFICATION, n_trials=40)

print("==================================================")
print(f" Best Expectancy: {study.best_value:.4f}")
print("==================================================")



 GPU Detected!
Loading Data...


[I 2025-11-27 20:42:44,981] A new study created in memory with name: no-name-ff47de3e-06b1-4b9b-9842-9e31514a1fc9



[Resume] Loaded 1 existing trials.

 Starting T0_L12_P1.0_S0.5_N30

 Pipeline Started... (Train Start: 2020-01-01)


/raid/invigoworks/anaconda3/lib/python3.10/site-packages/optuna/distributions.py:702: UserWarning: The distribution is specified by [180, 730] and step=180, but the range is not divisible by `step`. It will be replaced by [180, 720].
  warnings.warn(
/tmp/ipykernel_704681/324838015.py:116: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ta['EMA_72'] = ta.ema(close, length=72)


Valid Samples: 17806/17889 (Removed: 83)
Final Data Shape: (17806, 154)
Data Split Completed. Total 9 folds generated.

 Processing Fold 1 (walk_forward_rolling_reverse)
 Train Period: 2020-03-08 13:00:00 ~ 2022-02-26 09:00:00 (N=4320)
 Val   Period: 2022-02-28 13:00:00 ~ 2022-07-28 09:00:00 (N=900)
 Test  Period: 2022-07-30 13:00:00 ~ 2022-12-27 09:00:00 (N=900)
[Class Balance] Train Set: {0: 0.6518518518518519, 1: 0.34814814814814815}

[Feature Selection] Top 30 Features Selected:
 -> ATR_84, return_lag_6p, VIX_ma180_ratio, eth_chain_tvl_pct_chg_24h, VIX_pct_chg_24h, GOLD_ma180_ratio, SP500_ma180_ratio, MACDH_72_156_54, MFI_84, aave_eth_tvl_ma180_ratio, curve-dex_eth_tvl_1d_chg, PRICE_VS_LOW_30p, BREAKOUT_STR_30p, Dist_from_VWAP, return_lag_12p, Corr_ETH_BTC_24h, eth_chain_tvl_1d_chg_pct_chg_24h, eth_chain_tvl_1d_chg_ma180_ratio, uniswap_eth_tvl_1d_chg_pct_chg_24h, uniswap_eth_tvl_1d_chg_ma180_ratio, makerdao_eth_tvl_pct_chg_24h, aave_eth_tvl_pct_chg_24h, uniswap_eth_tvl_pct_chg_24h,

In [ ]:
# # lookahead = trial.suggest_int('lookahead', 12, 42, step=6)
# # p_mult = trial.suggest_float('profit_mult', 1.5, 3.0, step=0.1)
# # s_mult = trial.suggest_float('stop_mult', 0.8, 1.2, step=0.1)
# # top_n = trial.suggest_int('top_n', 15, 35, step=5)
# # train_days = trial.suggest_int('train_days', 365, 1095, step=365)
        
    
    
# df_merged = pd.read_csv("eth_4hour.csv")
# df_hour = pd.read_csv("eth_hour.csv")
# if 'timestamp' in df_merged.columns:
#     df_merged = df_merged.rename(columns={'timestamp': 'date'})

# df_merged['date'] = pd.to_datetime(df_merged['date'])
# df_merged = df_merged.sort_values('date').reset_index(drop=True)
# lookahead=12
# p_mult=1.5
# s_mult=0.8
# top_n=20
# train_days=365
# pipeline_result = build_complete_pipeline_corrected(
#                 df_raw=df_merged,
#                 df_hour=df_hour,
#                 train_start_date='2020-01-01',
#                 final_test_start='2025-01-01',
#                 lookahead_periods=lookahead,
#                 profit_mult=p_mult,
#                 stop_mult=s_mult,
#                 top_n=top_n,
#                 initial_train_days=train_days
#             )

In [ ]:
#pipeline_result